# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234414274647                   -0.50    7.0
  2   -7.250050799585       -1.81       -1.40    1.0
  3   -7.251235579267       -2.93       -1.99    4.0
  4   -7.251080541512   +   -3.81       -1.95    5.0
  5   -7.251328204018       -3.61       -2.63    4.0
  6   -7.251337776740       -5.02       -3.06    4.0
  7   -7.251338794807       -5.99       -4.18    3.0
  8   -7.251338796341       -8.81       -4.40    4.0
  9   -7.251338798671       -8.63       -5.20    3.0
 10   -7.251338798704      -10.48       -5.94    4.0
 11   -7.251338798704      -12.31       -6.48    2.0
 12   -7.251338798705      -13.37       -6.88    3.0
 13   -7.251338798705      -13.67       -7.57    2.0
 14   -7.251338798705      -14.45       -7.95    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06465593221737852
[ Info: Arnoldi iteration step 2: normres = 0.4424999383133998
[ Info: Arnoldi iteration step 3: normres = 0.960919207434664
[ Info: Arnoldi iteration step 4: normres = 0.3012283354229543
[ Info: Arnoldi iteration step 5: normres = 0.23729717463306063
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.15e-02, 2.52e-02, 1.43e-01, 1.85e-01, 2.37e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5394845587190044
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.73e-03, 1.02e-01, 5.15e-01, 8.71e-02, 7.18e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07411854409797758
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.09e-04, 3.18e-02, 5.69e-03, 1.86e-02, 5.66e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11272845303539494
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.98e-0